# Phase 3 실습: 실제 API 사용해보기

## 🎯 이 노트북에서 배울 것
- **Tavily API**를 실제로 호출하여 뉴스 검색하기
- **Gemini API**를 실제로 호출하여 텍스트 생성하기
- 다양한 파라미터를 바꿔가며 실험하기
- API 응답 구조 이해하기

## 📋 사전 준비
- `.env` 파일에 API 키 설정 완료
  - `TAVILY_API_KEY=tvly-xxxxx`
  - `GEMINI_API_KEY=xxxxx`
- 필요한 패키지 설치: `uv sync`

## ⚠️ 주의사항
- **API 호출은 무료 할당량을 소모합니다!**
- Tavily: 무료 플랜 월 1,000건
- Gemini: 분당 요청 제한 있음
- 불필요한 반복 실행은 피해주세요

## 📖 학습 순서
1. **환경 설정** - API 키 로드 및 확인
2. **Tavily API** - 뉴스 검색 실습
3. **Gemini API** - 텍스트 생성 실습
4. **통합 실습** - 검색 + 요약 파이프라인

---

# 1️⃣ 환경 설정

---

## 📚 API 키 로드하기

`.env` 파일에서 API 키를 불러옵니다.

`.env` 파일 예시:
```
TAVILY_API_KEY=tvly-xxxxxxxxxx
GEMINI_API_KEY=AIzaSyxxxxxxxxxx
```

In [ ]:
# ============================================
# 🔧 환경 설정: API 키 로드
# ============================================

import os
from dotenv import load_dotenv

# .env 파일 로드 (프로젝트 루트에서 실행 시)
load_dotenv()

# API 키 확인 (전체 키는 보안상 출력하지 않음)
tavily_key = os.getenv("TAVILY_API_KEY", "")
gemini_key = os.getenv("GEMINI_API_KEY", "")

print("=== API 키 확인 ===")
if tavily_key:
    print(f"✅ TAVILY_API_KEY: {tavily_key[:10]}...")
else:
    print("❌ TAVILY_API_KEY가 설정되지 않았습니다!")

if gemini_key:
    print(f"✅ GEMINI_API_KEY: {gemini_key[:10]}...")
else:
    print("❌ GEMINI_API_KEY가 설정되지 않았습니다!")

# 💡 API 키가 없으면 .env 파일을 확인하세요!

---

# 2️⃣ Tavily API 실습

---

## 📚 Tavily API란?

Tavily는 **AI 에이전트를 위한 검색 API**입니다.

- 공식 사이트: https://tavily.com/
- 무료 플랜: 월 1,000건 검색
- 뉴스 검색에 최적화된 `topic="news"` 모드 제공

### 주요 파라미터

| 파라미터 | 설명 | 예시 |
|---------|------|------|
| `query` | 검색 키워드 | "AI 기술" |
| `search_depth` | 검색 깊이 | "basic" 또는 "advanced" |
| `max_results` | 최대 결과 수 | 5, 10, 20 |
| `topic` | 검색 유형 | "general" 또는 "news" |
| `include_domains` | 특정 도메인만 검색 | ["bbc.com", "cnn.com"] |

In [ ]:
# ============================================
# 🔍 Tavily API 기본 사용법
# ============================================

from tavily import TavilyClient

# 클라이언트 초기화
tavily = TavilyClient(api_key=tavily_key)

# 기본 검색 실행
print("=== Tavily 기본 검색 ===")
print("검색어: AI 기술")
print("검색 중...\n")

response = tavily.search(
    query="AI 기술",
    max_results=3  # 결과 3개만
)

# 응답 구조 확인
print(f"응답 타입: {type(response)}")
print(f"응답 키: {response.keys()}")
print(f"결과 개수: {len(response.get('results', []))}")

In [ ]:
# ============================================
# 🔍 Tavily 응답 구조 자세히 보기
# ============================================

# 첫 번째 결과 자세히 보기
if response.get('results'):
    first_result = response['results'][0]
    
    print("=== 첫 번째 결과 구조 ===")
    print(f"키 목록: {first_result.keys()}")
    print()
    print(f"📌 title: {first_result.get('title', '없음')}")
    print(f"🔗 url: {first_result.get('url', '없음')}")
    print(f"📝 content: {first_result.get('content', '없음')[:100]}...")
    print(f"📅 published_date: {first_result.get('published_date', '없음')}")
    print(f"📊 score: {first_result.get('score', '없음')}")
else:
    print("결과가 없습니다.")

In [ ]:
# ============================================
# 🔍 Tavily 뉴스 모드 (topic="news")
# ============================================
# 뉴스에 특화된 검색 모드입니다.

print("=== 뉴스 모드 검색 ===")
print("검색어: 삼성전자")
print("모드: news\n")

news_response = tavily.search(
    query="삼성전자",
    topic="news",  # 뉴스 모드!
    max_results=5
)

print(f"검색 결과: {len(news_response.get('results', []))}건\n")

for i, result in enumerate(news_response.get('results', []), 1):
    title = result.get('title', '제목 없음')
    date = result.get('published_date', '날짜 없음')
    print(f"{i}. [{date}] {title[:50]}...")

In [ ]:
# ============================================
# 🔍 Tavily 고급 검색 (search_depth="advanced")
# ============================================
# 더 정확한 결과와 날짜 정보를 얻습니다.

print("=== 고급 검색 모드 ===")
print("검색어: 생성형 AI 트렌드")
print("깊이: advanced\n")

advanced_response = tavily.search(
    query="생성형 AI 트렌드",
    search_depth="advanced",  # 고급 검색!
    topic="news",
    max_results=5
)

print(f"검색 결과: {len(advanced_response.get('results', []))}건\n")

for i, result in enumerate(advanced_response.get('results', []), 1):
    title = result.get('title', '제목 없음')
    date = result.get('published_date', '날짜 없음')
    url = result.get('url', '')
    domain = url.split('/')[2] if url else '알 수 없음'
    print(f"{i}. [{date}] {title[:40]}...")
    print(f"   출처: {domain}\n")

In [ ]:
# ============================================
# 🔍 Tavily 특정 도메인만 검색 (include_domains)
# ============================================
# 신뢰할 수 있는 뉴스 사이트만 검색합니다.

print("=== 특정 도메인 검색 ===")
print("검색어: 경제 전망")
print("도메인: zdnet.co.kr, etnews.com\n")

domain_response = tavily.search(
    query="경제 전망",
    search_depth="advanced",
    topic="news",
    include_domains=["zdnet.co.kr", "etnews.com", "hani.co.kr"],  # 특정 도메인!
    max_results=5
)

print(f"검색 결과: {len(domain_response.get('results', []))}건\n")

for i, result in enumerate(domain_response.get('results', []), 1):
    title = result.get('title', '제목 없음')
    url = result.get('url', '')
    domain = url.split('/')[2] if url else '알 수 없음'
    print(f"{i}. {title[:50]}...")
    print(f"   🔗 {domain}\n")

## ✏️ 실습: Tavily API 직접 사용해보기

**목표**: 원하는 키워드로 뉴스를 검색하고 결과를 정리해보세요.

**힌트**:
- `query`: 검색하고 싶은 키워드
- `topic="news"`: 뉴스 모드
- `max_results`: 가져올 결과 수

In [ ]:
# ============================================
# ✏️ 실습: 나만의 검색
# ============================================

# TODO 1: 원하는 키워드를 입력하세요
my_keyword = "여기에 키워드 입력"  # 예: "반도체", "기후변화", "K-POP"

# TODO 2: 검색 실행
# my_response = tavily.search(
#     query=my_keyword,
#     topic="news",
#     search_depth="advanced",
#     max_results=5
# )

# TODO 3: 결과 출력 (제목, 날짜, 내용 요약)
# for i, result in enumerate(my_response.get('results', []), 1):
#     print(f"{i}. {result.get('title')}")
#     print(f"   날짜: {result.get('published_date')}")
#     print(f"   내용: {result.get('content', '')[:100]}...")
#     print()

---

# 3️⃣ Gemini API 실습

---

## 📚 Gemini API란?

Gemini는 **Google의 AI 모델**입니다.

- 공식 사이트: https://aistudio.google.com/
- 무료 플랜: 분당 요청 제한 있음
- 텍스트 생성, 요약, 번역 등 다양한 작업 가능

### 주요 모델

| 모델 | 설명 |
|------|------|
| `gemini-2.0-flash` | 빠르고 가벼운 모델 |
| `gemini-2.5-flash` | 성능과 속도의 균형 |
| `gemini-1.5-pro` | 고성능 모델 |

In [ ]:
# ============================================
# 🤖 Gemini API 기본 사용법
# ============================================

from google import genai

# 클라이언트 초기화
gemini = genai.Client(api_key=gemini_key)

# 사용할 모델
MODEL = "gemini-2.0-flash"  # 또는 "gemini-2.5-flash"

print("=== Gemini 기본 사용 ===")
print(f"모델: {MODEL}\n")

# 간단한 질문
response = gemini.models.generate_content(
    model=MODEL,
    contents="안녕하세요! 간단히 자기소개해주세요."
)

print("응답:")
print(response.text)

In [ ]:
# ============================================
# 🤖 Gemini 응답 구조 확인
# ============================================

print("=== Gemini 응답 구조 ===")
print(f"응답 타입: {type(response)}")
print(f"\n텍스트 결과:")
print(response.text[:200] if response.text else "없음")

# 응답에 포함된 정보
print(f"\n사용 토큰 정보:")
if hasattr(response, 'usage_metadata'):
    print(f"  - 입력 토큰: {response.usage_metadata.prompt_token_count}")
    print(f"  - 출력 토큰: {response.usage_metadata.candidates_token_count}")

In [ ]:
# ============================================
# 🤖 Gemini로 텍스트 요약하기
# ============================================

# 요약할 텍스트 (예시)
sample_text = """
인공지능(AI) 기술이 빠르게 발전하면서 다양한 산업 분야에서 혁신이 일어나고 있습니다. 
특히 생성형 AI는 텍스트, 이미지, 코드 생성 등 다양한 작업을 수행할 수 있어 주목받고 있습니다.
ChatGPT, Claude, Gemini 등 대형 언어 모델(LLM)들이 경쟁하며 성능을 높이고 있으며,
기업들은 이러한 AI를 업무에 도입하여 생산성을 향상시키고 있습니다.
그러나 AI의 발전과 함께 일자리 대체, 저작권 문제, 허위정보 생성 등의 우려도 제기되고 있습니다.
전문가들은 AI 기술의 책임있는 개발과 사용이 중요하다고 강조합니다.
"""

prompt = f"""
다음 글을 3줄로 요약해주세요:

{sample_text}
"""

print("=== 텍스트 요약 ===")
response = gemini.models.generate_content(
    model=MODEL,
    contents=prompt
)

print("원문 길이:", len(sample_text), "자")
print("\n요약 결과:")
print(response.text)

In [ ]:
# ============================================
# 🤖 Gemini로 뉴스 요약하기 (프로젝트 방식)
# ============================================
# 프로젝트에서 사용하는 것과 동일한 프롬프트 형식입니다.

# 가상의 뉴스 기사들
news_articles = [
    {"title": "삼성전자, 차세대 반도체 개발 성공", "content": "삼성전자가 3nm 공정 기반의 차세대 반도체 개발에 성공했다고 발표했다."},
    {"title": "SK하이닉스, AI 반도체 투자 확대", "content": "SK하이닉스가 AI 반도체 생산을 위해 10조원 규모의 투자를 발표했다."},
    {"title": "글로벌 반도체 수요 증가 전망", "content": "업계 전문가들은 AI 붐으로 인해 반도체 수요가 크게 증가할 것으로 전망했다."},
]

# 뉴스 컨텍스트 구성
news_context = ""
for i, article in enumerate(news_articles, 1):
    news_context += f"{i}. 제목: {article['title']}\n"
    news_context += f"   내용: {article['content']}\n\n"

# 프로젝트와 동일한 프롬프트
prompt = f"""
다음 뉴스 기사들의 핵심 내용을 한국어로 요약해주세요:
- 불릿 포인트 형식으로 최대 5개 항목
- 각 항목은 1~2문장

[뉴스 목록]
{news_context}
"""

print("=== 뉴스 요약 (프로젝트 방식) ===")
response = gemini.models.generate_content(
    model=MODEL,
    contents=prompt
)

print(response.text)

In [ ]:
# ============================================
# 🤖 Gemini 다양한 프롬프트 실험
# ============================================

# 실험 1: 번역
print("=== 실험 1: 번역 ===")
translate_response = gemini.models.generate_content(
    model=MODEL,
    contents="Translate to English: 인공지능이 세상을 바꾸고 있습니다."
)
print(translate_response.text)

# 실험 2: 감정 분석
print("\n=== 실험 2: 감정 분석 ===")
sentiment_response = gemini.models.generate_content(
    model=MODEL,
    contents="""다음 문장의 감정을 분석해주세요 (긍정/부정/중립):
    "오늘 날씨가 정말 좋아서 기분이 최고예요!"
    """
)
print(sentiment_response.text)

# 실험 3: 키워드 추출
print("\n=== 실험 3: 키워드 추출 ===")
keyword_response = gemini.models.generate_content(
    model=MODEL,
    contents="""다음 글에서 핵심 키워드 3개를 추출해주세요:
    "애플이 새로운 아이폰 16을 발표했습니다. 이번 모델은 AI 기능이 강화되었으며, 
    카메라 성능도 크게 향상되었습니다."
    """
)
print(keyword_response.text)

## ✏️ 실습: Gemini API 직접 사용해보기

**목표**: 다양한 프롬프트로 Gemini를 실험해보세요.

**아이디어**:
- 글 요약
- 질문 답변
- 텍스트 변환 (존댓말 ↔ 반말)
- 아이디어 생성

In [ ]:
# ============================================
# ✏️ 실습: 나만의 프롬프트
# ============================================

# TODO 1: 원하는 프롬프트를 작성하세요
my_prompt = "여기에 프롬프트 입력"

# TODO 2: API 호출
# my_response = gemini.models.generate_content(
#     model=MODEL,
#     contents=my_prompt
# )

# TODO 3: 결과 출력
# print(my_response.text)

---

# 4️⃣ 통합 실습: 검색 + 요약 파이프라인

---

## 📚 검색 → 요약 파이프라인

TrendTracker의 핵심 흐름을 직접 구현해봅니다:

```
[키워드 입력] → [Tavily로 검색] → [Gemini로 요약] → [결과 출력]
```

In [ ]:
# ============================================
# 🔄 전체 파이프라인 구현
# ============================================

def search_and_summarize(keyword: str, num_results: int = 5) -> dict:
    """
    키워드로 뉴스를 검색하고 AI로 요약합니다.
    
    Args:
        keyword: 검색 키워드
        num_results: 검색 결과 수
    
    Returns:
        dict: 검색 결과와 요약
    """
    result = {
        "keyword": keyword,
        "articles": [],
        "summary": "",
        "success": False
    }
    
    try:
        # 1단계: Tavily로 검색
        print(f"🔍 '{keyword}' 검색 중...")
        search_response = tavily.search(
            query=keyword,
            topic="news",
            search_depth="advanced",
            max_results=num_results
        )
        
        articles = search_response.get('results', [])
        if not articles:
            print("검색 결과가 없습니다.")
            return result
        
        print(f"✅ {len(articles)}건의 뉴스를 찾았습니다.")
        result["articles"] = articles
        
        # 2단계: 뉴스 컨텍스트 구성
        news_context = ""
        for i, article in enumerate(articles, 1):
            news_context += f"{i}. 제목: {article.get('title', '제목 없음')}\n"
            news_context += f"   내용: {article.get('content', '')[:200]}\n\n"
        
        # 3단계: Gemini로 요약
        print("🤖 AI가 요약하고 있습니다...")
        prompt = f"""
다음 뉴스 기사들의 핵심 내용을 한국어로 요약해주세요:
- 불릿 포인트 형식으로 최대 5개 항목
- 각 항목은 1~2문장

[뉴스 목록]
{news_context}
"""
        
        summary_response = gemini.models.generate_content(
            model=MODEL,
            contents=prompt
        )
        
        result["summary"] = summary_response.text
        result["success"] = True
        print("✅ 요약 완료!")
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
    
    return result

In [ ]:
# ============================================
# 🔄 파이프라인 실행
# ============================================

# 검색 + 요약 실행
result = search_and_summarize("인공지능 트렌드", num_results=3)

if result["success"]:
    print("\n" + "="*50)
    print(f"🔍 검색 키워드: {result['keyword']}")
    print("="*50)
    
    print("\n📰 검색된 뉴스:")
    for i, article in enumerate(result["articles"], 1):
        print(f"  {i}. {article.get('title', '제목 없음')[:50]}...")
    
    print("\n🤖 AI 요약:")
    print(result["summary"])

## ✏️ 실습: 나만의 검색 + 요약

**목표**: 원하는 키워드로 뉴스를 검색하고 요약해보세요.

In [ ]:
# ============================================
# ✏️ 실습: 나만의 검색 + 요약
# ============================================

# TODO: 원하는 키워드로 검색 + 요약 실행
# my_result = search_and_summarize("여기에 키워드", num_results=5)

# if my_result["success"]:
#     print("\n📰 검색된 뉴스:")
#     for i, article in enumerate(my_result["articles"], 1):
#         print(f"  {i}. {article.get('title')}")
#     
#     print("\n🤖 AI 요약:")
#     print(my_result["summary"])

---

# 🏆 챌린지

⚠️ **정답이 제공되지 않습니다.** 강사와 함께 풀어보세요!

---

In [ ]:
# ============================================
# 🏆 챌린지 1: 다양한 프롬프트 실험
# ============================================
# ⚠️ 정답이 제공되지 않습니다.

# TODO 1: 뉴스를 요약하는 대신, 키워드를 추출해보세요
# 프롬프트: "다음 뉴스들에서 핵심 키워드 5개를 추출해주세요: ..."


# TODO 2: 뉴스의 감정(긍정/부정)을 분석해보세요
# 프롬프트: "다음 뉴스들이 긍정적인지 부정적인지 분석해주세요: ..."


# TODO 3: 뉴스를 질문-답변 형태로 변환해보세요
# 프롬프트: "다음 뉴스 내용을 Q&A 형식으로 3개 만들어주세요: ..."


In [ ]:
# ============================================
# 🏆 챌린지 2: 고급 검색 파이프라인
# ============================================
# ⚠️ 정답이 제공되지 않습니다.

# TODO: compare_topics() 함수를 만드세요
# - 두 개의 키워드를 받아 각각 검색
# - 두 주제의 공통점과 차이점을 AI로 분석
# 
# 예시:
# compare_topics("삼성전자", "SK하이닉스")
# → 두 회사의 뉴스를 검색하고, 공통점/차이점 분석

# def compare_topics(topic1: str, topic2: str) -> str:
#     ...

In [ ]:
# ============================================
# 🏆 챌린지 3: 에러 처리 강화
# ============================================
# ⚠️ 정답이 제공되지 않습니다.

# TODO: safe_search_and_summarize() 함수를 만드세요
# - 모든 API 호출에 try-except 적용
# - 에러 종류에 따라 다른 메시지 출력
#   - API 키 오류: "API 키를 확인해주세요"
#   - 요청 한도 초과: "잠시 후 다시 시도해주세요"
#   - 네트워크 오류: "인터넷 연결을 확인해주세요"
# - 실패해도 프로그램이 죽지 않도록

# def safe_search_and_summarize(keyword: str) -> dict:
#     ...

---

# 📋 정리

## 이번 노트북에서 배운 것

| API | 용도 | 주요 파라미터 |
|-----|------|---------------|
| **Tavily** | 뉴스 검색 | `query`, `topic`, `search_depth`, `max_results` |
| **Gemini** | 텍스트 생성/요약 | `model`, `contents` (프롬프트) |

## Tavily 핵심 코드

```python
from tavily import TavilyClient

client = TavilyClient(api_key="...")
response = client.search(
    query="키워드",
    topic="news",
    search_depth="advanced",
    max_results=5
)
articles = response.get('results', [])
```

## Gemini 핵심 코드

```python
from google import genai

client = genai.Client(api_key="...")
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="프롬프트"
)
text = response.text
```

## ✅ 학습 체크리스트

- [ ] Tavily API로 뉴스를 검색할 수 있다
- [ ] Tavily의 다양한 파라미터를 알고 있다
- [ ] Gemini API로 텍스트를 생성할 수 있다
- [ ] 프롬프트를 다양하게 변형할 수 있다
- [ ] 검색 → 요약 파이프라인을 구현할 수 있다
- [ ] 챌린지를 강사와 함께 풀어봤다

---

🎉 **Phase 3 API 실습을 완료하셨습니다!**